# Setup

In [2]:
pip install sparql-dataframe


[notice] A new release of pip available: 22.1.2 -> 23.0
[notice] To update, run: python.exe -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install --quiet rdflib SPARQLWrapper pandas

Note: you may need to restart the kernel to use updated packages.


In [4]:
import rdflib
import ssl
import json
import pprint
import pandas as pd
from pprint import pprint
from SPARQLWrapper import SPARQLWrapper, JSON
import csv
from rdflib import URIRef
import requests
import sparql_dataframe

ssl._create_default_https_context = ssl._create_unverified_context
wikidata_endpoint = "https://query.wikidata.org/bigdata/namespace/wdq/sparql"
sparql_wd = SPARQLWrapper(wikidata_endpoint)
sparql_wd.setReturnFormat(JSON)

# Queries

**Gender Distribution of Serial Killers**
---

In [5]:
# Query 1: Total number of serial killers
query = """
SELECT (COUNT(DISTINCT ?person) as ?count) WHERE {
  ?person wdt:P31 wd:Q5.
  ?person wdt:P106 wd:Q484188.
}
"""

response = requests.get(
    "https://query.wikidata.org/sparql",
    params={"query": query, "format": "json"}
)

if response.status_code != 200:
    raise Exception("SPARQL query failed with status code " + str(response.status_code))

results = response.json()

total_count = int(results['results']['bindings'][0]['count']['value'])

# Query 2: Number of serial killers by gender
query = """
SELECT ?genderLabel (COUNT(?serialKiller) AS ?count) WHERE {
  ?serialKiller wdt:P106 wd:Q484188 ;
               wdt:P31 wd:Q5 ;
               wdt:P21 ?gender .
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en" .
    ?gender rdfs:label ?genderLabel .
  }
}
GROUP BY ?genderLabel
ORDER BY DESC(?count)
""" 

response = requests.get(
    "https://query.wikidata.org/sparql",
    params={"query": query, "format": "json"}
)

if response.status_code != 200:
    raise Exception("SPARQL query failed with status code " + str(response.status_code))

results = response.json()
genders = ['male', 'female', 'trans woman', 'intersex']
filtered_results = [result for result in results['results']['bindings'] if result['genderLabel']['value'].lower() in genders]
gender_count = {
    result['genderLabel']['value']: int(result['count']['value'])
    for result in filtered_results
}

# Create the DataFrame
df = pd.DataFrame({
    'Males': [gender_count.get('male', 0)],
    'Females': [gender_count.get('female', 0)],
    'Trans Women': [gender_count.get('trans woman', 0)],
    'Intersex': [gender_count.get('intersex', 0)],
    'Total': [total_count],
})

# Save the combined data to a CSV file
df.to_csv("serial_killers_stats.csv")

df


,Males,Females,Trans Women,Intersex,Total
0,947,134,2,1,1100


**Highest Serial Killer Death Counts**
---

In [ ]:
query = """
SELECT ?serialKillerLabel ?genderLabel ?victimCount ?countryOfCitizenshipLabel ?countryCode ?coordinates WHERE {
  ?serialKiller wdt:P106 wd:Q484188 .
  ?serialKiller wdt:P31 wd:Q5 .
  ?serialKiller wdt:P21 ?gender .
  ?serialKiller wdt:P1345 ?victimCount .
  ?serialKiller wdt:P27 ?countryOfCitizenship .
  ?countryOfCitizenship wdt:P297 ?countryCode .
  ?countryOfCitizenship wdt:P625 ?coordinates .
  SERVICE wikibase:label { 
    bd:serviceParam wikibase:language "en" .
    ?serialKiller rdfs:label ?serialKillerLabel .
    ?gender rdfs:label ?genderLabel .
    ?countryOfCitizenship rdfs:label ?countryOfCitizenshipLabel .
  }
}
ORDER BY DESC(?victimCount)
"""

# Create the DataFrame
def get_dataframe_from_sparql(query):
    sparql_wd.setQuery(query)
    sparql_wd.setReturnFormat(JSON)
    results = sparql_wd.query().convert()

    data = []
    for result in results["results"]["bindings"]:
     data.append({
     'Serial Killer': result['serialKillerLabel']['value'],
     'Gender': result['genderLabel']['value'],
     'Victims': result['victimCount']['value'],
     'Country': result['countryOfCitizenshipLabel']['value'],
     'Country Code': result['countryCode']['value'],
     'Coordinates': result['coordinates']['value']
        })

    df = pd.DataFrame(data)
    df[['Longitude', 'Latitude']] = df['Coordinates'].str.extract('(\-?\d+\.\d+)\s(\-?\d+\.\d+)')
    df = df.drop(columns=['Coordinates'])
    df.columns = ['Serial Killer', 'Gender', 'Victims', 'Country', 'Country Code', 'Longitude', 'Latitude']
    return df

df = get_dataframe_from_sparql(query)

# Save the data to a CSV file
df.to_csv("serial_killers_most_victims.csv")

df


,Serial Killer,Gender,Victims,Country,Country Code,Longitude,Latitude
0,Samuel Little,male,93,United States of America,US,-98.5795,39.828175
1,Gary Ridgway,male,49,United States of America,US,-98.5795,39.828175
2,Salvatore Riina,male,45,Italy,IT,12.5,42.5
3,Wang Qiang,male,45,People's Republic of China,CN,103.451944444,35.844722222
4,Andrei Chikatilo,male,43,Russia,RU,94.25,66.416666666
...,...,...,...,...,...,...,...
68,Joji Obara,male,2,Japan,JP,136.0,35.0
69,Carol M. Bundy,female,2,United States of America,US,-98.5795,39.828175
70,Richard Rogers (serial killer),male,2,United States of America,US,-98.5795,39.828175
71,Mark Errin Rust,male,2,Australia,AU,137.0,-28.0


**What Country has the Most Serial Killers?**
---

In [10]:
query = """
SELECT DISTINCT ?coordinates ?countryLabel ?countryCode (COUNT(?serial_killer) AS ?count)(SUM(IF(?gender = wd:Q6581097, 1, 0)) AS ?male_count) (SUM(IF(?gender = wd:Q6581072, 1, 0)) AS ?female_count) ?population WHERE {
  ?serial_killer wdt:P106 wd:Q484188 .
  ?serial_killer wdt:P31 wd:Q5 .
  ?serial_killer wdt:P27 ?country .
  ?country wdt:P625 ?coordinates .
  ?country wdt:P298 ?countryCode .
  ?country rdfs:label ?countryLabel .
  OPTIONAL { ?serial_killer wdt:P21 ?gender }
  OPTIONAL { ?country wdt:P1082 ?population }
  FILTER (LANG(?countryLabel) = "en")
  FILTER (?country != wd:Q117)
}
GROUP BY ?countryLabel ?countryCode ?coordinates ?population
ORDER BY DESC(?count)
"""

# Create the DataFrame
def get_dataframe_from_sparql(query):
    sparql_wd.setQuery(query)
    sparql_wd.setReturnFormat(JSON)
    results = sparql_wd.query().convert()

    data = []
    for result in results["results"]["bindings"]:
        data.append({
       'Coordinates': result['coordinates']['value'],
       'Country': result['countryLabel']['value'],
       'Country Code': result['countryCode']['value'],
       'Male': result['male_count']['value'],
       'Female': result['female_count']['value'],
       'Total': int(result['count']['value']),
       'Population': int(result['population']['value'])
        })

    df = pd.DataFrame(data)
    df['Rate per 100,000'] = (df['Total'] / df['Population']) * 100000
    df[['Longitude', 'Latitude']] = df['Coordinates'].str.extract('(\-?\d+\.\d+)\s(\-?\d+\.\d+)')
    df = df.drop(columns=['Coordinates'])
    df.columns = ['Country', 'Country Code', 'Male', 'Female', 'Total', 'Population', 'Rate per 100,000', 'Longitude', 'Latitude']
    return df

df = get_dataframe_from_sparql(query)

# Save the data to a CSV file
df.to_csv("serial_killer_countries.csv")

df

,Country,Country Code,Male,Female,Total,Population,"Rate per 100,000",Longitude,Latitude
0,United States of America,USA,239,39,280,331449281,0.084477,-98.5795,39.828175
1,Soviet Union,SUN,68,2,70,293047571,0.023887,90.0,65.0
2,Russia,RUS,55,3,58,146804372,0.039508,94.25,66.416666666
3,France,FRA,43,6,49,67749632,0.072325,2.0,47.0
4,United Kingdom,GBR,36,5,43,67326569,0.063868,-2.0,54.6
...,...,...,...,...,...,...,...,...,...
66,Dominican Republic,DOM,1,0,1,10403761,0.009612,-70.2,18.8
67,Israel,ISR,1,0,1,9187000,0.010885,35.0,31.0
68,Afghanistan,AFG,1,0,1,37466414,0.002669,66.0,33.0
69,Bangladesh,BGD,1,0,1,165775000,0.000603,89.866667,24.016667


**Where Is a Serial Killer Likely to Work?**
---

In [74]:
query = """
SELECT ?occupationLabel (SUM(IF(?gender = wd:Q6581097, 1, 0)) AS ?maleCount) (SUM(IF(?gender = wd:Q6581072, 1, 0)) AS ?femaleCount) (SUM(IF(?gender = wd:Q1097630, 1, 0)) AS ?intersexCount) (COUNT(?serialKiller) AS ?total) WHERE {
  ?serialKiller wdt:P106 wd:Q484188;
               wdt:P106 ?occupation;
               wdt:P21 ?gender .
  ?occupation wdt:P31 ?profession.
  FILTER (?profession = wd:Q3922583 || ?profession = wd:Q28640 || ?profession = wd:Q192581 || ?profession = wd:Q4164871 || ?profession = wd:Q6958747 || ?profession = wd:Q1914636)
  SERVICE wikibase:label {
    bd:serviceParam wikibase:language "en".
    ?occupation rdfs:label ?occupationLabel .
  }
}
GROUP BY ?occupationLabel 
ORDER BY DESC(?total)
"""

# Create the DataFrame
def get_dataframe_from_sparql(query):
    sparql_wd.setQuery(query)
    sparql_wd.setReturnFormat(JSON)
    results = sparql_wd.query().convert()

    data = []
    for result in results["results"]["bindings"]:
        data.append({
       'Occupation': result['occupationLabel']['value'],
       'Male': result['maleCount']['value'],
       'Female': result['femaleCount']['value'],
       'Intersex': result['intersexCount']['value'],
       'Total': result['total']['value']
        })

    df = pd.DataFrame(data)
    df.columns = ['Occupation', 'Male', 'Female', 'Intersex', 'Total']
    return df

df = get_dataframe_from_sparql(query)

# Save the data to a CSV file
df.to_csv("serial_killers_occupation.csv")

df

,Occupation,Male,Female,Intersex,Total
0,nurse,14,17,0,31
1,military personnel,25,0,0,25
2,physician,16,2,0,18
3,prostitute,3,5,0,8
4,writer,7,1,0,8
5,soldier,8,0,0,8
6,scientist,8,0,0,8
7,journalist,8,0,0,8
8,police officer,8,0,0,8
9,mechanic,6,0,0,6


**What Is the Punishment for Serial Killers?**
---

In [11]:
query = """
SELECT DISTINCT ?penaltyLabel (SUM(IF(?gender = wd:Q6581097, 1, 0)) AS ?maleCount) (SUM(IF(?gender = wd:Q6581072, 1, 0)) AS ?femaleCount) (COUNT(DISTINCT ?person) AS ?count) WHERE {
  ?person wdt:P106 wd:Q484188 ; 
          wdt:P1596 ?penalty ;
          wdt:P21 ?gender .
  ?penalty rdfs:label ?penaltyLabel .
  FILTER (LANG(?penaltyLabel) = "en")
}
GROUP BY ?penaltyLabel 
ORDER BY DESC(?count)
"""

# Create the DataFrame
def get_dataframe_from_sparql(query):
    sparql_wd.setQuery(query)
    sparql_wd.setReturnFormat(JSON)
    results = sparql_wd.query().convert()

    data = []
    for result in results["results"]["bindings"]:
        data.append({
       'Penalty': result['penaltyLabel']['value'],
       'Male': int(result['maleCount']['value']),
       'Female': int(result['femaleCount']['value']),
       'Total': int(result['count']['value']),
        })

    df = pd.DataFrame(data)
    df.columns = ['Penalty', 'Male', 'Female', 'Total']
    return df

df = get_dataframe_from_sparql(query)

# Save the data to a CSV file
df.to_csv("serial_killers_penalty.csv")

df

,Penalty,Male,Female,Total
0,capital punishment,133,15,148
1,life imprisonment,102,9,111
2,involuntary commitment,7,1,8
3,incarceration,5,1,6
4,capital punishment in Japan,3,0,3
5,hanging,1,1,2
6,back-to-back life sentences,1,0,1
7,solitary confinement,1,0,1
8,electric chair,1,0,1
9,None,1,0,1


**Educated Serial Killers. Which Serial Killers Actually Earned Graduate Degrees?**
--- 

In [9]:
# Statistics on serial killers with and without higher education (university)
query = """
SELECT 
  (COUNT(DISTINCT ?personWithEducation) AS ?countWithEducation)
  (COUNT(DISTINCT ?personWithoutEducation) AS ?countWithoutEducation)
WHERE {
  {
    SELECT DISTINCT ?person
    WHERE {
      ?person wdt:P106 wd:Q484188 ;
              wdt:P69 ?education .
    }
  }
  BIND (COUNT(DISTINCT ?person) AS ?total) .
  {
    SELECT DISTINCT ?personWithEducation
    WHERE {
      ?personWithEducation wdt:P106 wd:Q484188 ;
                          wdt:P69 ?education .
      ?education wdt:P31/wdt:P279* wd:Q3918 .
    }
  }
  BIND (COUNT(DISTINCT ?personWithEducation) AS ?withEducation) .
  {
    SELECT DISTINCT ?personWithoutEducation
    WHERE {
      ?personWithoutEducation wdt:P106 wd:Q484188 ;
                             wdt:P69 ?education .
      FILTER NOT EXISTS { ?education wdt:P31/wdt:P279* wd:Q3918 . }
    }
  }
  BIND (COUNT(DISTINCT ?personWithoutEducation) AS ?withoutEducation) .
}
"""

sparql_wd.setQuery(query)

results = sparql_wd.query().convert()

# Create the DataFrame
df = pd.DataFrame([
    {
        'With Higher Education' : result['countWithEducation']['value'],
        'Without Higher Education' : result['countWithoutEducation']['value'],
    }
    for result in results['results']['bindings']
])

# Save the data to a CSV file
df.to_csv("serial_killers_education_stats.csv")

df

,With Higher Education,Without Higher Education
0,39,91


In [12]:
query = """
SELECT ?killerLabel (GROUP_CONCAT(DISTINCT ?countryLabel; separator=", ") AS ?country) (GROUP_CONCAT(DISTINCT ?educationLabel; separator=", ") AS ?education) (GROUP_CONCAT(DISTINCT ?occupationLabel; separator=", ") AS ?occupation) (GROUP_CONCAT(DISTINCT ?victimLabel; separator=", ") AS ?victims) (GROUP_CONCAT(DISTINCT ?conditionLabel; separator=", ") AS ?condition)  WHERE {
?killer wdt:P106 wd:Q484188 .
?killer wdt:P69 ?education .
?education wdt:P31/wdt:P279* wd:Q3918 .
OPTIONAL {
?killer wdt:P106 ?occupation .
?occupation wdt:P31 ?profession .
FILTER (?profession = wd:Q3922583 || ?profession = wd:Q28640 || ?profession = wd:Q192581 || ?profession = wd:Q4164871 || ?profession = wd:Q6958747 || ?profession = wd:Q1914636)
}
OPTIONAL {
?killer wdt:P1345 ?victim .
}
OPTIONAL {
?killer wdt:P1050 ?condition .
?condition wdt:P1995 ?healthSpecialty .
FILTER (?healthSpecialty IN (wd:Q7867, wd:Q9418, wd:Q170912, wd:Q178059))
}
OPTIONAL {
?killer wdt:P27 ?country .
?country wdt:P31 wd:Q3624078 .
}
SERVICE wikibase:label {
bd:serviceParam wikibase:language "en" .
?killer rdfs:label ?killerLabel .
?education rdfs:label ?educationLabel .
?occupation rdfs:label ?occupationLabel .
?victim rdfs:label ?victimLabel .
?condition rdfs:label ?conditionLabel .
?country rdfs:label ?countryLabel .
}
}
GROUP BY ?killerLabel
"""

# Create the DataFrame
def get_dataframe_from_sparql(query):
    sparql_wd.setQuery(query)
    sparql_wd.setReturnFormat(JSON)
    results = sparql_wd.query().convert()

    data = []
    for result in results["results"]["bindings"]:
        data.append({
        'Serial Killer': result["killerLabel"]["value"],
        'Country': result["country"]["value"],
        'University': result["education"]["value"],
        'Occupation': result["occupation"]["value"],
        'Victims': result["victims"]["value"],
        'Mental Condition': result["condition"]["value"]
        })

    df = pd.DataFrame(data)
    df.columns = ['Serial Killer', 'Country', 'University', 'Occupation', 'Victims', 'Mental Condition']
    return df

df = get_dataframe_from_sparql(query)

# Save the data to a CSV file
df.to_csv("serial_killers_education.csv")

df

,Serial Killer,Country,University,Occupation,Victims,Mental Condition
0,Andrei Chikatilo,Russia,"Russian University of Transport, Rostov State ...","military personnel, schoolteacher",43,"pedophilia, borderline personality disorder"
1,Eric Beishline,,Purdue University,,,
2,Li Yijiang,,Beijing Institute of Technology,,,
3,James Dale Ritchie,,West Virginia University,,,
4,Robert Neal Rodriguez,,Brigham Young University,missionary,,
5,Ted Bundy,United States of America,"University of Utah, University of Washington, ...",psychologist,36,antisocial personality disorder
6,Ted Kaczynski,United States of America,"University of Michigan, Harvard University","writer, mathematician, environmentalist",,
7,Jeffrey Dahmer,United States of America,Ohio State University,"confectioner, soldier",17,"personality disorder not otherwise specified, ..."
8,Harold Shipman,United Kingdom,University of Leeds,physician,,
9,H. H. Holmes,United States of America,University of Vermont,"physician, pharmacist, hotel manager",,


**Dangerous Minds: The Mental Illnesses Of Infamous Criminals**
--- 

In [78]:
query = """
SELECT ?conditionLabel (SUM(IF(?gender = wd:Q6581097, 1, 0)) AS ?maleCount) (SUM(IF(?gender = wd:Q6581072, 1, 0)) AS ?femaleCount) (COUNT(?condition) AS ?count) WHERE {
  ?person wdt:P106 wd:Q484188 .
  ?person wdt:P1050 ?condition .
  ?person wdt:P21 ?gender .
  ?condition wdt:P1995 ?healthSpecialty .
FILTER (?healthSpecialty IN (wd:Q7867, wd:Q9418, wd:Q170912, wd:Q178059))
SERVICE wikibase:label {
    bd:serviceParam wikibase:language "en" .
    ?condition rdfs:label ?conditionLabel .
    ?occupation rdfs:label ?occupationLabel .
  }
}
GROUP BY ?conditionLabel
ORDER BY DESC(?count)
"""

def get_dataframe_from_sparql(query):
    sparql_wd.setQuery(query)
    sparql_wd.setReturnFormat(JSON)
    results = sparql_wd.query().convert()

    data = []
    for result in results["results"]["bindings"]:
      data.append({
        'Mental Ilness' : result['conditionLabel']['value'],
        'Male': int(result['maleCount']['value']),
        'Female': int(result['femaleCount']['value']),
        'Total' : result['count']['value']
        })

    df = pd.DataFrame(data)
    df.columns = ['Mental Ilness', 'Male', 'Female', 'Total']
    return df

df = get_dataframe_from_sparql(query)

# Save the data to a CSV file
df.to_csv("serial_killers_mental_conditions.csv")

df

,Mental Ilness,Male,Female,Total
0,antisocial personality disorder,18,0,18
1,borderline personality disorder,6,0,6
2,pedophilia,4,0,4
3,schizophrenia,2,1,3
4,bipolar disorder,3,0,3
5,personality disorder not otherwise specified,2,0,2
6,alcoholism,2,0,2
7,necrophilia,2,0,2
8,kleptomania,2,0,2
9,personality disorder,2,0,2
